# Regression

## Objectives

### Answer business requirement 1: 
* Using a predictive model to **determine the current Reaming Useful Life (RUL) of any given replaceable part** (in this case an industrial air filter).


## Inputs

* outputs/datasets/transformed/dfTransformedTotal.csv

## Outputs

* Train set (features and target)
* Test set (features and target)
* Validation set (features and target)
* ML pipeline to predict RUL
* Labels map
* Feature Importance Plot



---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

# Feature Engineering
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.model_selection import train_test_split

# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    median_absolute_error
    )

# ML algorithms
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor


df_total = pd.read_csv(f'outputs/datasets/transformed/dfTransformedTotal.csv')
frame = df_total['Data_No'].iloc[0:len(df_total)]
df_train = df_total[frame < 51].reset_index(drop=True)
df_test = df_total[frame > 50].reset_index(drop=True)
df_total

Extract bins that reach **600 pa** of differential pressure or more in **df_train** dataset

In [ ]:
dp_total = df_train['Differential_pressure'].map(float).round(decimals=4)
df_train['Differential_pressure'] = dp_total
n = df_train['Differential_pressure'][0:len(df_train)]
df_train_dp = df_train[n >= 600].reset_index(drop=True)
RUL_extract = df_train_dp['Data_No']
RUL_additional = df_train.loc[df_train['Data_No'].isin(RUL_extract)]
RUL_additional

Include **additional RUL** variables that have a fully completed test cycle to **increase the total data in the modelling dataframe**
Remove NaN Values

In [ ]:
df = pd.concat([df_test, RUL_additional], ignore_index=True)
print(df_train.shape, '= df_train')
print(df_test.shape, '= df_test')
print(df.shape, '= df')
df.sort_values('Data_No', ascending=True)

# MP Pipeline: Regressor

## Convert Ordinal Numbers into Categorical Values
The target and all requirements are already in a numerical format (float and integer) from our previous engineering steps. 
* **Notwithstanding**; we will convert the **dust type** back into a categorical variable to demonstrate the inclusion of a categorical encoder in each pipeline.
* We will also take the opportunity to remove **data number** from the regression set. 
    * This variable is a category and may confound the results as each RUL measure is within a series of data bins of 'not always complete' tests.

In [ ]:
# data_no = df['Data_No'].map(str)
# df.drop(['Data_No'], axis=1)
dust = df['Dust'].map(str)
df['Dust'] = dust
df = df.drop(['Data_No'], axis=1)
df.info()

## Create ML pipeline

In [ ]:

def PipelineOptimization(model):
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['Dust'])),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(
                                                        variables=['Differential_pressure', '4point_EWM', 'log_EWM',
                                                                'Flow_rate', 'Time', 'Dust_feed',
                                                                'change_DP', 'change_EWM', 'mass_g',
                                                                'cumulative_mass_g', 'Tt','filter_balance'],
                                                        method="spearman",
                                                        threshold=0.6,
                                                        selection_method="variance")),
        ("feat_scaling", StandardScaler()),
        ("feat_selection",  SelectFromModel(model)),
        ("model", model),
    ])
    return pipeline_base


Custom Class for hyperparameter optimisation

In [ ]:
# from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score (R²)'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score (R²)': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score (R²)', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches


## Split Train, Test and Validation Sets

In [ ]:
# from sklearn.model_selection import train_test_split

X_working, X_test, y_working, y_test = train_test_split(
    df.drop(['RUL'], axis=1),
    df['RUL'],
    test_size=0.25,
    random_state=8,
    shuffle=True
)

X_train, X_validate, y_train, y_validate = train_test_split(
    X_working,
    y_working,
    test_size=0.25,
    random_state=8,
    shuffle=True
)

print('\n', X_train.shape, y_train.shape, '= Train set\n',
      X_validate.shape, y_validate.shape, '= Validation set\n',
      X_test.shape, y_test.shape, '= Test set\n',
      '===========\n',
      df.shape[0], '= Total Observations\n')


In [ ]:
X_train

## Grid Search CV - Sklearn

### Use default hyperparameters to find most suitable algorithm

In [ ]:
models_quick_search = {
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    'LinearRegression': LinearRegression(),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "SGDRegressor": SGDRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    "AdaBoostRegressor": {},
    "DecisionTreeRegressor": {},
    "ExtraTreesRegressor": {},
    "GradientBoostingRegressor": {},
    'LinearRegression': {},
    "RandomForestRegressor": {},
    "SGDRegressor": {},
    "XGBRegressor": {},
}

Do a hyperparameter optimisation search using default hyperparameters

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Define the Color Map

In [ ]:
import matplotlib as mpl
cmap = mpl.colormaps['viridis']
# cmap = plt.cm.RdBu

Check results

In [ ]:
# import matplotlib.pyplot as plt

grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score (R²)')
results = grid_search_summary['mean_score (R²)']
results.plot(kind="bar",title="Mean Scores (R²)")

axes = plt.gca()
axes.set_ylim([0.0,1.05])
plt.xticks(rotation=0, fontsize=8)

plt.title("Mean Score (R²) of various Regression Model's\n(using all variables)")
plt.ylabel('Mean Score (R²)\n')
plt.xlabel('\nModel Index No.')
plt.show()
grid_search_summary

The average **R² score** (mean_score) indicates how well a model of the data fits the actual data. 

We note that
* From the **original 6 features**, plus an additional **8 calculated** ones, produces an almost perfect prediction of remaining useful life (RUL).
    * R² score ranges from **0.74** to **0.99**, which is exceptional, as value of 1 represents a perfect fit.
    * This is result is exceptional, however unusual and requires further investigation.
    * The natural inter-correlation of the calculated requirements may be influencing the models score, so we will exclude these for further review.
* The **Random Forest Regressor** looks to be the best performing model among the 7 reviewed at this stage.

---

## Exclude Calculated Requirements
These are naturally be cross correlated to the base requirement they are calculated and may unduly skew the model.

In [ ]:
df.head(3)

Consolidate columns required in **Dataframe**, **Train**, **Test** & **Validation** Sets

In [ ]:
columns_req = ['Differential_pressure', 'Flow_rate', 'Time', 'Dust_feed', 'Dust', 'RUL']
df = df.filter(columns_req)
X_train = X_train.filter(columns_req)
X_validate = X_validate.filter(columns_req)
X_test = X_test.filter(columns_req)

print('\n', X_train.shape, y_train.shape, '= Train set\n',
      X_validate.shape, y_validate.shape, '= Validate set\n',
      X_test.shape, y_test.shape, '= Test set\n',
      '===========\n',
      df.shape[0], '= Total Observations\n')
      
df.head(3)

## Save for Later
We will take the opportunity to save this **hybrid dataframe** for use in the **feature study** section that looks to answer **business requirement 2**.

In [ ]:
df_export = df.copy()
dust_density = ['ISO 12103-1, A2 Fine Test Dust' if n == '0.9' else ('ISO 12103-1, A3 Medium Test Dust' if n == '1.025' else 'ISO 12103-1, A4 Coarse Test Dust') for n in df_export['Dust']]
df_export['Dust'] = dust_density
df_export.to_csv(f'outputs/datasets/transformed/dfCombinedHybrid.csv',index=False)
df_export.head(3)

Simple check to see all dust values have been converted

In [ ]:
df_export.Dust.unique().reshape(-1).tolist()

## Re-Define the Pipeline

In [ ]:
def PipelineOptimization(model):
    pipeline_base = Pipeline([

        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['Dust'])),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(
                                                        variables=['Differential_pressure',
                                                                'Flow_rate', 'Time', 'Dust_feed'],
                                                        method="spearman",
                                                        threshold=0.6,
                                                        selection_method="variance")),
        ("feat_scaling", StandardScaler()),
        ("feat_selection",  SelectFromModel(model)),
        ("model", model),
    ])

    return pipeline_base

Re-Run hyperparameter optimization search using default hyperparameters **on less variables**.

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check Results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score (R²)')
results = grid_search_summary['mean_score (R²)']
results.plot(kind="bar",title="Mean Scores (R²)")

axes = plt.gca()
axes.set_ylim([0,1.1])
plt.xticks(rotation=0, fontsize=8)

plt.title("Mean Score (R²) of various Regression Model's\n(using original variables only)")
plt.ylabel('Mean Score (R²)\n')
plt.xlabel('\nModel Index No.')
plt.show()
grid_search_summary

### Observations
* From the **original 6 features** maintains the almost perfect prediction of remaining useful life (RUL) for most regression models.
    * R² score ranges from ±**0.45** to ±**0.95**.
    * We see a slight reduction of the scores across all tests, which is understandable considering the removal of possibly cross correlated calculated variables.
    * There is negligible difference between the top 3 ranked models.

* The **Random Forest Regressor** remains the best performing model with an R² of **0.948985**.
    * This is result is exceptional.
    * High R² Scores are unusual and will require further investigation.

* The R² score of the top 5 ranked estimators is much higher than the **0.7** tolerance we decided in the business case.
    * We could use this information to feedback to the business team to review the business model.
    * A tolerance level between **0.85** to **0.95** or higher may be suitable for this dataset / business case.
    * At the high performance levels seen in a variety of models, the **speed of calculating each model** may also be a further consideration for the business team.

## Optimal **hyperparameter configuration** of the most suitable model
Here we will perform an extensive grid search on the most suitable model to find the optimal combination of hyper-parameters.

First step is to define the model and parameters for the extensive search

#### Random Forest Regressor (12min)

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

models_search = {
    'RandomForestRegressor': RandomForestRegressor(),
}

params_search = {
    'RandomForestRegressor':{
        # 'model__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        # 'model__criterion': ['squared_error', 'friedman_mse', 'poisson'],
        'model__criterion': ['poisson'],
        # # 'model__max_depth': [None],
        # 'model__max_depth': [3,10,None],
        'model__max_features': [1.0, 'sqrt', 'log2'],
        # 'model__n_estimators': [100,300,600,29089],
        'model__n_estimators': [100,400,800],
        # 'model__n_jobs': [None, 1],
        # 'model__n_jobs': [None],
    }
}

Extensive GridSearch CV

In [ ]:
search_regr = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search_regr.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search_regr.score_summary(sort_by='mean_score (R²)')
grid_search_summary

---

#### Extra Trees Regressor (48min)

In [ ]:
# # documentation to help on hyperparameter list: 
# # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html

# models_search = {
#     'ExtraTreesRegressor': ExtraTreesRegressor(),
# }

# params_search = {
#     'ExtraTreesRegressor':{
#         'model__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
#         # # 'model__max_depth': [None],
#         # 'model__max_depth': [3,10,None],
#         # 'model__max_features': [1.0, 'sqrt', 'log2'],
#         # model__min_samples_split': [2,4,6],
#         # 'model__n_estimators': [100,200,300],
#         # 'model__n_jobs': [None, 1],
#     }
# }

Extensive GridSearch CV

In [ ]:
# search_et = HyperparameterOptimizationSearch(models=models_search, params=params_search)
# search_et.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=2)

Check Results

In [ ]:
# grid_search_summary_ExtraTrees, grid_search_pipelines_ExtraTrees = search_et.score_summary(sort_by='mean_score (R²)')
# grid_search_summary_ExtraTrees

Concatenation into a summary

In [ ]:
# grid_search_summary = pd.concat([grid_search_summary_RForest, grid_search_summary_ExtraTrees], ignore_index=True)
# grid_search_pipelines = dict(grid_search_summary_RForest); grid_search_pipelines.update(grid_search_summary_ExtraTrees)

---

#### Check the best model

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

Hyperparameters for best model

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

Define the best regressor, based on search

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

## Assess feature importance

Recall best Parameters

In [ ]:
best_parameters

Manually define these into the best model

In [ ]:
reg_model = RandomForestRegressor(
    criterion='poisson',
    max_features='sqrt',
    n_estimators=800,
    )
reg_model.fit(X_train, y_train)

Visualize results

In [ ]:
feat_importances = (pd.Series(reg_model.feature_importances_, index=X_train.columns)
                    .nlargest(6)
                    .plot(kind='bar'))
plt.xticks(fontsize=8)
plt.title('Feature Importance\n')
plt.ylabel("F-Score\n")
plt.xlabel('\nFeature')
plt.show()

#### Observations
* From the 6 original features, we dropped `Data_No` as it is a catagorical variable that arbitrarily describes the test number and has no relation to the patterns seen in the dataset.
* Among the remaing 5 variables, 2 show higher relevance to predict Remaining Useful Life (RUL) that the others
    * `Dust Feed` and `Differential Pressure`

---

## Evaluate Regressor Performance on Train and Test Sets

Compute a performance metric on the data held out for testing, **df_test**
* [R² score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) (also called Coefficient of Determination)
* [Mean Absolute Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) (MAE)
* [Median Absolute Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.median_absolute_error.html) (MdAE)
* [Mean Squared Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE)
* Root Mean Squared Error (RMSE).

We could also consider:
* Almost Correct Predictions Error Rate (ACPER)
* Mean Absolute Percentage Error (MAPE) and 
* Adjusted R² Score 
    * _((1 - R²) * (sample_size - 1)) * -1 / (sample_size - no_independent_features - 1))_

Define Evaluation Functions

In [ ]:
# from sklearn.metrics import (
#     r2_score, mean_squared_error, mean_absolute_error,
#     median_absolute_error
#     )
# import numpy as np
# import seaborn as sns

def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)


def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R² Score:', r2_score(y, prediction).round(4))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(4))
    print('Median Absolute Error:', median_absolute_error(y, prediction).round(4))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(4))
    print('Root Mean Squared Error:', np.sqrt(
        mean_squared_error(y, prediction)).round(4))
    print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])
    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])
    axes[0].set_xlabel("Actual")
    axes[0].set_ylabel("Predictions")
    axes[0].set_title("Train Set")

    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])
    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])
    axes[1].set_xlabel("Actual")
    axes[1].set_ylabel("Predictions")
    axes[1].set_title("Test Set")

    plt.show()


Run Performance Evaluation

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

In [ ]:
best_regressor_pipeline

#### Observations
* The pipeline performance (R² Score) Train set: ±**0.97** and Test set: ±**0.95**.
* The represents a very high performance of the model to predict remaining useful life.
* This is much higher than the current business requirement is an R² Score of 0.7 or higher.
* Our hyperparameter combination exceeds our performance criteria.

Additionally:
* The predictions tend to follow the actual values.
* We initially added more hyperparameters in the extensive search.
* Optimal hyperparameter combinations were chosen to train all possible models more quickly.
* We see a few outliers in the supplied dataset that tend to mirror each other, reflecting the sourcing of train, test data from the same data bins.

#### Considerations
* Due to the high performance of this model, additional hyperparameters are not warranted to increase performance in this case.
* We could replace the **feature selection step** in the model pipeline for a **PCA (Principal Component Analysis) step** to select variables according to the magnitude (from largest to smallest in absolute values) of their coefficients (loadings).
    * In this case, we already have a small number of attributes and performance exceeds the current business case requirement, so a PCA is not warranted, however;
    * To **demonstrate the process** we will perform a PCA and **highlight any changes** that occur in performance.

Next:
* Refit our ML Pipeline with a PCA.


---

# Regressor with PCA

Review PCA separately to the scaled data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

pipeline = Pipeline([('scaler', MinMaxScaler()), ('regressor', RandomForestRegressor())])
pipeline.fit(X_train, y_train)

r2 = pipeline.score(X_test, y_test)
print(f'RandomForrestRegression (defaults): {r2}') # RFR: 0.9997308011141385

In [ ]:
pipeline = Pipeline([('scaler', MinMaxScaler()), ('regressor', RandomForestRegressor(
    criterion='poisson',
    max_features='sqrt',
    n_estimators=800,
))])
pipeline.fit(X_train, y_train)

r2 = pipeline.score(X_test, y_test)
print(f'RandomForrestRegression (custom x 3): {r2}') # 0.999761456617107

* All components explain ±**99%** of the data 
* Just 3 of these components also explain **99%** of the data

Apply PCA separately to the scaled data

In [ ]:
# pipeline = PipelineOptimization(model=RandomForestRegressor(random_state=0))
# pipeline_pca = Pipeline(pipeline.steps[:4])
# # df_pca = pipeline_pca.fit_transform(df.drop(['Data_No'], axis=1))
# df_pca = pipeline_pca.fit_transform(X_train, y_train)

In [ ]:
# # import numpy as np
# # import seaborn as sns
# # from sklearn.decomposition import PCA

# n_components = 3

# def pca_components_analysis(df_pca, n_components):
#     pca = PCA(n_components=n_components).fit(df_pca)
#     x_PCA = pca.transform(df_pca)  # array with transformed PCA

#     ComponentsList = ["Component " + str(number)
#                       for number in range(n_components)]
#     dfExplVarRatio = pd.DataFrame(
#         data=np.round(100 * pca.explained_variance_ratio_, 3),
#         index=ComponentsList,
#         columns=['Explained Variance Ratio (%)'])

#     dfExplVarRatio['Accumulated Variance'] = dfExplVarRatio['Explained Variance Ratio (%)'].cumsum(
#     )

#     PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum(
#     )

#     print(
#         f"* The {n_components} components explain {round(PercentageOfDataExplained,4)}% of the data \n")
#     plt.figure(figsize=(12, 5))
#     sns.lineplot(data=dfExplVarRatio,  marker="o")
#     plt.xticks(rotation=90)
#     plt.yticks(np.arange(0, 110, 10))
#     plt.show()


# pca_components_analysis(df_pca=df_pca, n_components=n_components)


In [ ]:
# n_components = 2
# pca_components_analysis(df_pca=df_pca, n_components=n_components)

## Rewrite ML Pipeline for Modelling

In [ ]:
from sklearn.decomposition import PCA

n_components = 3

def PipelineOptimization(model):
    pipeline_base = Pipeline([
        
        # ("filter_and_split", filter_and_split(df)),

        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['Dust'])),
                                                     
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(
                                                        variables=['Differential_pressure',
                                                                'Flow_rate', 'Time', 'Dust_feed'],
                                                        method="spearman",
                                                        threshold=0.6,
                                                        selection_method="variance")),
        ("feat_scaling", StandardScaler()),
        # PCA replace Feature Selection
        # ("feat_selection",  SelectFromModel(model)),
        ("PCA", PCA(n_components=n_components, random_state=0)),
        ("model", model),
    ])

    return pipeline_base

## Grid Search CV – Sklearn

In [ ]:
print('Summary:\n', X_train.shape, y_train.shape, '= Train set\n', X_test.shape, y_test.shape, '= Test set')

### Use standard hyperparameters to find the most suitable model.

In [ ]:
models_quick_search = {
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    'LinearRegression': LinearRegression(),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "SGDRegressor": SGDRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    "AdaBoostRegressor": {},
    "DecisionTreeRegressor": {},
    "ExtraTreesRegressor": {},
    "GradientBoostingRegressor": {},
    'LinearRegression': {},
    "RandomForestRegressor": {},
    "SGDRegressor": {},
    "XGBRegressor": {},
}

Do a quick optimisation search 

In [ ]:
quick_search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
quick_search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_PCA, grid_search_pipelines_PCA = quick_search.score_summary(sort_by='mean_score (R²)')
grid_search_summary_PCA

### Do an extensive search on the most suitable model to find the best hyperparameter configuration.

Define model and parameters for extensive search

In [ ]:
# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

models_search = {
    'RandomForestRegressor': RandomForestRegressor(),
}

params_search = {
    'RandomForestRegressor':{
        # 'model__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        # 'model__criterion': ['squared_error', 'friedman_mse', 'poisson'],
        'model__criterion': ['poisson'],
        # # 'model__max_depth': [None],
        # 'model__max_depth': [3,10,None],
        'model__max_features': [1.0, 'sqrt', 'log2'],
        # 'model__n_estimators': [100,300,600,29089],
        'model__n_estimators': [100,400,800],
        # 'model__n_jobs': [None, 1],
        # 'model__n_jobs': [None],
    }
}

Extensive GridSearch CV

In [ ]:
search_PCA = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search_PCA.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary_PCA, grid_search_pipelines_PCA = search_PCA.score_summary(sort_by='mean_score (R²)')
grid_search_summary_PCA

Check the best model

In [ ]:
best_model = grid_search_summary_PCA.iloc[0,0]
best_model

Parameters for best model

In [ ]:
grid_search_pipelines_PCA[best_model].best_params_

Define the best regressor

In [ ]:
best_regressor_pipeline = grid_search_pipelines_PCA[best_model].best_estimator_
best_regressor_pipeline

Visualize most important features

In [ ]:
reg_model = RandomForestRegressor(
    criterion='poisson',
    max_features='sqrt',
    n_estimators=800,
    )
reg_model.fit(X_train, y_train)

In [ ]:
feat_importances = (pd.Series(reg_model.feature_importances_, index=X_train.columns)
                    .nlargest(6)
                    .plot(kind='bar'))
plt.xticks(fontsize=8)
plt.title('Feature Importance\n')
plt.ylabel("F-Score\n")
plt.xlabel('\nFeature')
plt.show()

## Evaluate Regressor on Train and Tests Sets

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

# Which pipeline to choose?

We fitted the following pipelines:
* Random Forest Regression (with all variables)
* Random Forest Regression (with original 6 variables)
* Random Forest Regression with PCA
<!-- * Classifier -->

### Observations
All the regressor pipelines exceeded the expected performance threshold (0.7 R² score) for the train and test set.
The 

The Importance of features changed between Regression vs Regression + PCA processes:
* 3 pipeline components explain more than 90% of the data and improves the performance of the model.
* The `max_features` component changes from `log3` to `sqrt` improving all measures of performance.
* The `n_components` component changed from `800` to `400` improving all measures of performance.
* The R² Score is moderately improved.
* Error rates significantly decrease across both **train** and **test** sets.

|| Performance Measure | Regressor | Regressor + PCA |
|---|---|---|---|
|**Train Set**|R² Score:|± 0.97|± 0.99|
||Mean Absolute Error:|± 6.66|± 1.75|
||Median Absolute Error:|± 3.70|± 0.71|
||Mean Squared Error:|± 111.80|± 9.62|
||Root Mean Squared Error:|± 10.57|± 3.10|
|||||
|**Test Set**|R² Score:|± 0.95|± 0.98|
||Mean Absolute Error:|± 9.49|± 4.46|
||Median Absolute Error:|± 5.48|± 1.81|
||Mean Squared Error:|± 227.77|± 61.71|
||Root Mean Squared Error:|± 15.09|± 7.86|

**Future Features**:
* Future models may be tuned was tuned on Remaining Useful Life accross 3 Dust classes.
    * This may assist us to detect the remaining useful life relative to the size of dust particles for business case 2
<!-- * It has strong performance for class A4 (<4 months) and class A2 (+20 months) -->
<!-- * It has reasonable performance for class A2 (<4 months) and class A2 (+20 months) -->
<!-- * It has weak performance for class A3 (<4 months) and class A2 (+20 months) -->

In [ ]:
# pipeline_clf
best_regressor_pipeline

# Refit pipeline with best features

## Rewrite Pipeline

In [ ]:
n_components = 2

def PipelineOptimization(model):
    pipeline_base = Pipeline([
        # ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
        #                                              variables=['Dust'])),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(
                                                        variables=['Differential_pressure','Dust_feed', 'Time'],
                                                        method="spearman",
                                                        threshold=0.6,
                                                        selection_method="variance")),
        ("feat_scaling", StandardScaler()),
         # ("feat_selection",  SelectFromModel(model)),
        ("PCA", PCA(n_components=n_components, random_state=0)),
        ("model", model),
    ])
    return pipeline_base


## Consolidate Dataset, Train, Test and Validation Set, only with best features

In [ ]:
columns_req = ['Differential_pressure', 'Flow_rate', 'Time', 'Dust_feed', 'Dust']
df = df.filter(columns_req)
X_train = X_train.filter(columns_req)
X_validate = X_validate.filter(columns_req)
X_test = X_test.filter(columns_req)

print('\n', X_train.shape, y_train.shape, '= Train set\n',
      X_validate.shape, y_validate.shape, '= Validate set\n',
      X_test.shape, y_test.shape, '= Test set\n',
      '===========\n',
      df.shape[0], '= Total Observations\n')
      
X_train.head(3)

Subset Best Features

In [ ]:
best_model

In [ ]:
best_regressor_pipeline

In [ ]:
reg_model = RandomForestRegressor(
    criterion='poisson',
    max_features='sqrt',
    n_estimators=800,
    )
reg_model.fit(X_train, y_train)
feat_importances = (pd.Series(reg_model.feature_importances_, index=X_train.columns)
                    .nlargest(6)
                    .plot(kind='bar'))

Consolidate **Train**, **Test** and **Validation** data by top 3 feature importance

In [ ]:
n_features = 3
best_features = pd.Series(reg_model.feature_importances_, index=X_train.columns).nlargest(n_features).index.to_list()
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)
X_validate = X_validate.filter(best_features)

print('Summary:\n', X_train.shape, y_train.shape,'= Train set\n',
        X_validate.shape, y_validate.shape, '= Validation set\n',
        X_test.shape, y_test.shape, '= Test set\n')
X_train.head(6)

In [ ]:
best_features

## Grid Search CV – Sklearn

Define Pipeline

In [ ]:
n_components = 2

def PipelineOptimization(model):
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['Dust'])),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(
                                                        # variables=['Differential_pressure','Dust_feed', 'Time'],
                                                        variables=['Differential_pressure','Dust_feed'],
                                                        method="spearman",
                                                        threshold=0.6,
                                                        selection_method="variance")),
        ("feat_scaling", StandardScaler()),
        #  ("feat_selection",  SelectFromModel(model)),
        ("PCA", PCA(n_components=n_components, random_state=0)),
        ("model", model),
    ])
    return pipeline_base

We are using the same model from the previous GridCV search

In [ ]:
models_search

And the best parameters from the previous GridCV search

In [ ]:
best_parameters

Include manually

In [ ]:
models_search = {
    'RandomForestRegressor': RandomForestRegressor(),
}

params_search = {
    'RandomForestRegressor':{
        'model__criterion': ['poisson'],
        'model__max_features': ['sqrt'],
        'model__n_estimators': [800],
    }
}

GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)


Check results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score (R²)')
grid_search_summary

Check the best model

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

Define the best clf pipeline

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

## Assess feature importance

In [ ]:
reg_model = RandomForestRegressor(
    criterion='poisson',
    max_features='sqrt',
    n_estimators=800,
    )
reg_model.fit(X_train, y_train)
df_feature_importance = pd.Series(reg_model.feature_importances_, index=X_train.columns).nlargest(n_features).index.to_list()
print(f'The {len(best_features)} most important features in descending order. \n'
      f'\nThe above model was trained on the following variables: \n{df_feature_importance}')
feat_importances = (pd.Series(reg_model.feature_importances_, index=X_train.columns)
                    .nlargest(6)
                    .plot(kind='bar'))


# Save files to the repo

We will generate the following files to include in the app

* Train set
* Test set
* Validation set
* Modeling pipeline
* Features importance plot

In [ ]:
import joblib
import os

version = 'v2'
file_path = f'outputs/ml_pipeline/predict_rul/{version}'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

## Train Set: features and target

In [ ]:
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

## Test Set: features and target

In [ ]:
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

## Validation Set: features and target

In [ ]:
X_validate.head()

In [ ]:
X_validate.to_csv(f"{file_path}/X_validate.csv", index=False)

In [ ]:
y_validate

In [ ]:
y_validate.to_csv(f"{file_path}/y_validate.csv", index=False)

## Modelling pipeline

ML pipeline for predicting RUL

In [ ]:
best_regressor_pipeline

In [ ]:
joblib.dump(value=best_regressor_pipeline, filename=f"{file_path}/RandomForestRegressor_pipeline.pkl")

## Feature importance plot

In [ ]:
feat_importances = (pd.Series(reg_model.feature_importances_, index=X_train.columns)
                    .nlargest(6)
                    .plot(kind='bar'))
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')

C'est Fini

---